<a href="https://colab.research.google.com/github/faynercosta/faynercosta/blob/main/Poker_Player.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we need to preprocess the data from the log files into a format that can be used to train the deep learning model. We will create a function to parse the log files and extract relevant information, such as players' hole cards, community cards, and actions taken by the players.

Here's a function to parse the log files:

In [2]:
import os
import re

def parse_log_files(folder_path):
    data = []

    for file in os.listdir(folder_path):
        if file.endswith(".log"):
            with open(os.path.join(folder_path, file), "r") as f:
                for line in f:
                    if line.startswith("STATE"):
                        match = re.match(
                            r"STATE:(\d+):([\w\d\/]+):([\w\d|\/]+):([\d\-\|]+):([\w\|]+)",
                            line.strip(),
                        )
                        if match:
                            hand_idx, actions, cards, money, names = match.groups()
                            data.append(
                                {
                                    "hand_idx": int(hand_idx),
                                    "actions": actions,
                                    "cards": cards,
                                    "money": list(map(int, money.split("|"))),
                                    "names": names.split("|"),
                                }
                            )
    return data

# Replace 'root_folder' with the actual path of the folder containing the log files
log_data = parse_log_files("raw_data")


Now that we have the data in a structured format, we can preprocess it further by converting the cards and actions into numerical representations. To do this, we can create a dictionary for card ranks and suits, and another dictionary for actions:

In [5]:
RANKS = {'2': 0, '3': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 6, '9': 7, 'T': 8, 'J': 9, 'Q': 10, 'K': 11, 'A': 12}
SUITS = {'c': 0, 'd': 1, 'h': 2, 's': 3}
ACTIONS = {'f': 0, 'c': 1, 'r': 2}

def convert_card_to_numeric(card):
    rank, suit = card[0], card[1]
    return 4 * RANKS[rank] + SUITS[suit]

def convert_action_to_numeric(action):
    return ACTIONS[action[0]]

def preprocess_data(log_data):
    preprocessed_data = []
    for hand in log_data:
        preprocessed_hand = {
            "hand_id": hand["hand_idx"],
            "actions": [convert_action_to_numeric(a) for a in hand["actions"] if a in ACTIONS],
            "hole_cards": [list(map(convert_card_to_numeric, cards.split('|'))) for cards in hand["cards"].split('/')[0].split('|')],
            "community_cards": list(map(convert_card_to_numeric, re.findall(r'[\w\d]{2}', hand["cards"].split('/')[1]))) if len(hand["cards"].split('/')) > 1 else [],
            "money": hand["money"],
            "names": hand["names"],
        }
        preprocessed_data.append(preprocessed_hand)
    return preprocessed_data

preprocessed_data = preprocess_data(log_data)



In [15]:
print((preprocessed_data[:3]))

[{'hand_id': 0, 'actions': [0, 0, 2, 0, 2, 0, 0], 'hole_cards': [[37], [12], [44], [8], [48], [18]], 'community_cards': [], 'money': [325, -100, 0, 0, -225, 0], 'names': ['MrPink', 'Eddie', 'MrOrange', 'Bill', 'MrBlue', 'Pluribus']}, {'hand_id': 1, 'actions': [0, 2, 0, 0, 1, 0, 1, 2, 0], 'hole_cards': [[35], [27], [3], [10], [22], [42]], 'community_cards': [17, 41, 16], 'money': [-225, -100, 0, 325, 0, 0], 'names': ['Eddie', 'MrOrange', 'Bill', 'MrBlue', 'Pluribus', 'MrPink']}, {'hand_id': 2, 'actions': [0, 0, 2, 0, 2, 0, 2, 2, 1], 'hole_cards': [[51], [38], [16], [19], [46], [17]], 'community_cards': [22, 49, 24], 'money': [50, -100, 0, 0, 50, 0], 'names': ['MrOrange', 'Bill', 'MrBlue', 'Pluribus', 'MrPink', 'Eddie']}]


In [11]:
print(inputs)

[]


The next step is to create a deep learning model using advanced techniques to play Texas Hold'em Poker. To achieve this, we will use a recurrent neural network (RNN) with LSTM (Long Short-Term Memory) cells to process the sequential data. We will use the Keras library, which is part of TensorFlow, to build and train the model. First, let's import the necessary libraries:

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


Now, let's prepare the data for the model. We need to create input and output data from the preprocessed data. The input data will consist of the actions, hole cards, community cards, and money, while the output data will be the next action taken by the model:

In [34]:
max_action_length = max([len(hand["actions"]) for hand in preprocessed_data])

def create_dataset(preprocessed_data, max_action_length, max_community_cards):
    X = []
    y = []

    for hand in preprocessed_data:
        for i in range(len(hand["actions"]) - 1):
            padded_actions = hand["actions"][:i + 1] + [-1] * (max_action_length - (i + 1))
            flattened_hole_cards = [card for sublist in hand["hole_cards"] for card in sublist]
            padded_community_cards = hand["community_cards"] + [-1] * (max_community_cards - len(hand["community_cards"]))
            input_data = (
                padded_actions +
                flattened_hole_cards +
                padded_community_cards +
                hand["money"]
            )
            output_data = hand["actions"][i + 1]

            X.append(input_data)
            y.append(output_data)

    return np.array(X, dtype=object), np.array(y, dtype=object)

max_community_cards = max(len(hand["community_cards"]) for hand in preprocessed_data)
X, y = create_dataset(preprocessed_data, max_action_length, max_community_cards)


In [35]:
#Now, we need to normalize the input data and split it into training and testing sets:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Now that we have the preprocessed train and test data, let's create a deep learning model to play Texas Hold'em Poker using TensorFlow and Keras. We'll use a neural network architecture that leverages LSTM layers to handle the sequential nature of the data.

First, let's import the necessary libraries.

Then,let's create the model:



In [44]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout


In [45]:
def build_poker_model(input_shape, num_actions):
    model = Sequential()
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_actions, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

num_actions = 3
input_shape = (1, X_train.shape[1])
poker_model = build_poker_model(input_shape, num_actions)


In [46]:
#Now, let's reshape the training and testing data for input into the LSTM layers:
X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


In [48]:
#adjusting label types
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

#Training the model:
epochs = 10
batch_size = 64

history = poker_model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test), epochs=epochs, batch_size=batch_size)


Epoch 1/10
1016/1016 [==============================] - 25s 20ms/step - loss: 0.7870 - accuracy: 0.6631 - val_loss: 0.7406 - val_accuracy: 0.6794
Epoch 2/10
1016/1016 [==============================] - 22s 21ms/step - loss: 0.7091 - accuracy: 0.6939 - val_loss: 0.6850 - val_accuracy: 0.7030
Epoch 3/10
1016/1016 [==============================] - 21s 21ms/step - loss: 0.6726 - accuracy: 0.7071 - val_loss: 0.6624 - val_accuracy: 0.7106
Epoch 4/10
1016/1016 [==============================] - 19s 18ms/step - loss: 0.6524 - accuracy: 0.7140 - val_loss: 0.6331 - val_accuracy: 0.7196
Epoch 5/10
1016/1016 [==============================] - 19s 19ms/step - loss: 0.6315 - accuracy: 0.7190 - val_loss: 0.6114 - val_accuracy: 0.7300
Epoch 6/10
1016/1016 [==============================] - 22s 22ms/step - loss: 0.6153 - accuracy: 0.7232 - val_loss: 0.6024 - val_accuracy: 0.7228
Epoch 7/10
1016/1016 [==============================] - 19s 18ms/step - loss: 0.6057 - accuracy: 0.7255 - val_loss: 0.5933 -

In [49]:
loss, accuracy = poker_model.evaluate(X_test_reshaped, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


508/508 [==============================] - 4s 8ms/step - loss: 0.5784 - accuracy: 0.7334
Test Loss: 0.5784344673156738
Test Accuracy: 0.7334153652191162
